# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('output', exist_ok=True)

## Question 1: Basic GroupBy Operations

### Part 1.1: Load and Explore Data

**TODO: Load and merge the datasets**

In [9]:
# TODO: Load the three CSV files into dataframes
provider_df = pd.read_csv('data/provider_data.csv')
facility_df = pd.read_csv('data/facility_data.csv')
encounter_df = pd.read_csv('data/encounter_data.csv')

# TODO: Print the shapes of each dataframe
print("Provider data shape:", provider_df.shape)
print("Facility data shape:", facility_df.shape)
print("Encounter data shape:", encounter_df.shape)

# TODO: Merge the dataframes for analysis
# Hint: Start by merging encounter_df with provider_df on 'provider_id'
#       Then merge the result with facility_df on 'facility_id'
# Note: Both encounter_df and provider_df have 'facility_id', and both encounter_df and facility_df have 'region'
#       After merging, you may need to handle duplicate column names (pandas will add suffixes like _x and _y)
#       You can drop one of the duplicate columns and rename if needed
merged_df = encounter_df.merge(provider_df, on='provider_id')
merged_df = merged_df.drop(columns=['facility_id_y']).rename(columns={'facility_id_x': 'facility_id'})
merged_df = merged_df.merge(facility_df, on='facility_id')
merged_df = merged_df.drop(columns=['region_y']).rename(columns={'region_x': 'region'})

# TODO: Print the merged dataframe shape, columns, and first few rows
print("\nMerged data shape:", merged_df.shape)
print("\nColumns:", merged_df.columns)
print("\nFirst few rows:")
print(merged_df.head())

Provider data shape: (500, 7)
Facility data shape: (10, 6)
Encounter data shape: (5000, 12)

Merged data shape: (5000, 21)

Columns: Index(['encounter_id', 'patient_id', 'provider_id', 'facility_id',
       'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code',
       'service_charge', 'insurance_paid', 'patient_paid', 'region',
       'provider_name', 'provider_type', 'specialty', 'years_experience',
       'license_number', 'facility_name', 'facility_type', 'beds',
       'established_date'],
      dtype='object')

First few rows:
  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_c

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [52]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
# TODO: Calculate mean, sum, count for years_experience by facility
# TODO: Calculate total encounter charges by facility
# TODO: Calculate total insurance payments by facility
# TODO: Find the facility with highest total encounter charges
summary1 = merged_df.groupby('facility_id').agg(
    {
        'years_experience' : ['mean', 'sum', 'count'],
        'service_charge' : ['sum'],
        'insurance_paid' : ['sum']
    }
).reset_index()

display(summary1.sort_values(by=(('service_charge', 'sum')), ascending=False).head(1))
# TODO: Create a summary dataframe with one row per facility showing:
#       facility_name, total_providers, avg_years_experience, total_service_charge, total_insurance_paid
summary2 = pd.DataFrame()
summary2['total_providers'] = merged_df.groupby('facility_id')['provider_id'].agg('count')
summary2['avg_years_experience'] = merged_df.groupby('facility_id')['years_experience'].agg('mean')
summary2['total_service_charge'] = merged_df.groupby('facility_id')['service_charge'].agg('sum')
summary2['total_insurance_paid'] = merged_df.groupby('facility_id')['insurance_paid'].agg('sum')
summary2 = summary2.merge(facility_df[['facility_id', 'facility_name']], on='facility_id')

display(summary2)
# TODO: Save the summary dataframe as 'output/q1_groupby_analysis.csv'
summary2.to_csv("output/q1_groupby_analysis.csv", index=False)

facility_id years_experience              service_charge insurance_paid
                          mean    sum count            sum            sum
6      FAC007        19.811676  10520   531      361811.04      291790.95

,facility_id,total_providers,avg_years_experience,total_service_charge,total_insurance_paid,facility_name
0,FAC001,492,19.563008,295508.13,236798.50,City General Hospital
1,FAC002,510,20.160784,334992.21,269207.92,Community Health Clinic
2,FAC003,478,20.209205,316284.98,251750.43,Regional Medical Center
3,FAC004,489,20.137014,297283.06,236839.71,Northside Urgent Care
4,FAC005,441,19.319728,276658.83,222139.04,Cardiology Specialty Center
5,FAC006,509,19.381139,334927.30,267183.95,Oncology Treatment Center
6,FAC007,531,19.811676,361811.04,291790.95,Pediatric Care Center
7,FAC008,495,19.668687,310735.25,248439.20,Emergency Medical Center
8,FAC009,505,20.102970,323315.71,260297.44,Primary Care Clinic
9,FAC010,550,20.778182,349909.18,278503.38,Surgical Specialty Center


### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [44]:
# TODO: Add facility mean years_experience as new column
# TODO: Add facility standard deviation of years_experience
# TODO: Create normalized years_experience (z-score within facility)
# TODO: Add facility total encounter charges as new column
merged_df['year_experience_Mean'] = merged_df.groupby('facility_id')['years_experience'].transform('mean')
merged_df['Salary_Std'] = merged_df.groupby('facility_id')['years_experience'].transform('std')
merged_df['Salary_Normalized'] = merged_df.groupby('facility_id')['years_experience'].transform(lambda x: (x - x.mean()) / x.std())
merged_df['Total_encounter_charges'] = merged_df.groupby('facility_id')['service_charge'].transform('sum')

# TODO: Display the enhanced dataframe
display(merged_df.head())
# TODO: Save results as 'output/q1_aggregation_report.txt'
merged_df.to_csv("output/q1_aggregation_report.txt", index=False)

,encounter_id,patient_id,provider_id,facility_id,encounter_date,encounter_type,diagnosis_code,procedure_code,service_charge,insurance_paid,...,years_experience,license_number,facility_name,facility_type,beds,established_date,year_experience_Mean,Salary_Std,Salary_Normalized,Total_encounter_charges
0,ENC00001,PAT01564,PR0259,FAC004,2023-08-21,Office Visit,M79.3,99213,345.03,290.33,...,11,LIC000259,Northside Urgent Care,Urgent Care,313,1994-10-04,20.137014,10.581439,-0.863495,297283.06
1,ENC00002,PAT00296,PR0258,FAC001,2023-09-11,Inpatient,N18.9,85025,550.48,482.51,...,19,LIC000258,City General Hospital,Urgent Care,276,1999-11-27,19.563008,10.702137,-0.052607,295508.13
2,ENC00003,PAT01381,PR0213,FAC001,2023-03-20,Lab,K21.9,80053,149.84,116.67,...,3,LIC000213,City General Hospital,Urgent Care,276,1999-11-27,19.563008,10.702137,-1.547636,295508.13
3,ENC00004,PAT01131,PR0146,FAC004,2023-09-03,Outpatient Procedure,H52.9,70450,1468.48,1198.71,...,6,LIC000146,Northside Urgent Care,Urgent Care,313,1994-10-04,20.137014,10.581439,-1.336020,297283.06
4,ENC00005,PAT01388,PR0354,FAC003,2023-06-14,Emergency,G43.9,99214,672.23,507.22,...,36,LIC000354,Regional Medical Center,Urgent Care,459,1982-05-10,20.209205,11.575935,1.364105,316284.98


## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [ ]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
# TODO: Filter facilities with average years_experience > 10 (experienced staff)
# TODO: Filter facilities with total encounter charges > 250000 (high volume)

# TODO: Create a summary of filtered results
# TODO: Save results as 'output/q2_filter_analysis.csv'

### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [ ]:
# TODO: Create custom function to calculate provider statistics
def provider_stats(group):
    # TODO: Return mean, std, min, max, range for years_experience
    pass

# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)

# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers in a group by encounter volume"""
    # TODO: Return top n rows by encounter count, selecting only provider_id, provider_name, 
    #       years_experience, specialty
    pass

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = merged_df.groupby('facility_name').apply(
    lambda x: top_providers(x, n=2)
).reset_index(level=0)  # Moves facility_name from index to column

# TODO: Save results as 'output/q2_performance_report.txt'

### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [ ]:
# TODO: Group by facility type and region
# TODO: Calculate statistics for each facility_type-region combination
# TODO: Use unstack to convert to wide format
# TODO: Use stack to convert back to long format

# TODO: Analyze the hierarchical structure
# TODO: Save results as 'output/q2_hierarchical_analysis.csv'

## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [ ]:
# TODO: Create pivot table: encounter charges by procedure code and region
# TODO: Create pivot table with multiple aggregations (sum, mean, count)
# TODO: Add totals (margins) to pivot table
# TODO: Handle missing values with fill_value

# Tip: When saving a pivot table to CSV, the index (row labels) is preserved by default.
#      This is usually desired - when reading back, use index_col=0 to restore the index.
# TODO: Save results as 'output/q3_pivot_analysis.csv'

### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [ ]:
# TODO: Create crosstab of facility type vs region with counts
# TODO: Create crosstab with margins (row and column totals)
# TODO: Create multi-dimensional crosstab showing facility type × encounter type × region combinations
#       (Hint: use pd.crosstab with multiple index/column parameters, or create manually from the data)

# TODO: Analyze the cross-tabulation results
# TODO: Save results as 'output/q3_crosstab_analysis.csv'

### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [ ]:
# TODO: Create heatmap from pivot table
# TODO: Create bar chart from pivot table
# TODO: Customize colors and styling
# TODO: Add appropriate titles and labels

# TODO: Save the plot as 'output/q3_pivot_visualization.png'

## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques